In [314]:
#Fundamentals
import os 
import sys
import numpy as np
import pandas as pd
np.random.seed(55)

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline 

#Machine Learning Algorithms

import sklearn
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Multiprocessing
from multiprocessing.dummy import Pool as ThreadPool


In [315]:
#Versions
print('python', sys.version.split(' ')[0])
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('sklearn', sklearn.__version__)

os.getcwd()

python 3.7.7
numpy 1.18.1
pandas 1.0.3
sklearn 0.22.1


'C:\\Users\\HyewonKwak\\PycharmProjects\\Data'

In [316]:
#Encoding Check
print(sys.getdefaultencoding())

utf-8


In [317]:
#Load datasets
sample_submission = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')


In [318]:
#직관적인 시각화를 위해서 Indication Dictionary
indicator_dict = {
    'length' : ['rho'],
    'source' : ['650_src','660_src','670_src','680_src','690_src','700_src','710_src','720_src','730_src','740_src','750_src','760_src'
               ,'780_src','790_src','800_src','810_src','820_src','830_src','840_src','850_src','860_src','870_src','880_src','890_src',
               '900_src','910_src','920_src','930_src','940_src','950_src','960_src','970_src','980_src','990_src'],
    'detect' : ['650_dst',
               '660_dst', '670_dst', '680_dst', '690_dst', '700_dst', '710_dst',
               '720_dst', '730_dst', '740_dst', '750_dst', '760_dst', '770_dst',
               '780_dst', '790_dst', '800_dst', '810_dst', '820_dst', '830_dst',
               '840_dst', '850_dst', '860_dst', '870_dst', '880_dst', '890_dst',
               '900_dst', '910_dst', '920_dst', '930_dst', '940_dst', '950_dst',
               '960_dst', '970_dst', '980_dst', '990_dst']
    }

In [319]:
#train 데이터에서 예측해야할 hbo2 hbb ca na

train_X=train[train.columns[:71]].copy()
train_Y=train[train.columns[71:]].copy()


test_X = test.copy()

combined_X = pd.concat([train_X,test_X],axis=0)
X_list = [train_X,test_X]

In [320]:
# 데이터 전처리하기
print(combined_X.isnull().sum().sum())
print(train_X.isnull().sum().sum())
print(train_Y.isnull().sum().sum())

140168
70168
0


In [321]:
#아웃라이어 제거
#def del_outlier(x):
#    from_out = x.mean() - 3 * x.std()
#    to_out = x.mean() + 3 * x.std()
    
#    x = x[(x>from_out) & (x<to_out)]
    
#    return x#
#train_X=del_outlier(train_X)

#test_X=del_outlier(test_X)

In [322]:
src_columns = [k for k in train_X.columns if 'src' in k]
dst_columns = [k for k in train_X.columns if 'dst' in k]
od_columns = [k for k in train_X.columns if 'od' in k]

for X in X_list:
    X_dst=X[dst_columns]
    X[dst_columns]=X_dst.interpolate(axis=1)
    X.fillna(method='bfill',inplace=True)


In [325]:
for X in X_list:
    X['i0']=np.sum(X[X.columns[X.columns.str.contains('src')]],axis=1)
    X['i1']=np.sum(X[X.columns[X.columns.str.contains('dst')]],axis=1)
    X[subtract_col]=np.subtract(X[src_columns],X[dst_columns])
    X['re1']=-np.log((X['i0']-X['i1'])/X['i0'])#Od
    X['re2']=X['i0']-X['i1']
    
    

In [326]:


for X in X_list:
    #흡수계수
    X['S_ac'] = np.log(X['rho']*X['rho']*X['re2'])/X['rho']
    #위상천이변화율
    X['S_pi'] =X['re1']/X['rho']
    
    X['ab']=X['S_ac']/X['S_pi']-X['S_pi']/X['S_ac'] 
    X['di']=((X['S_ac']*X['S_ac'])-(X['S_pi']*X['S_pi']))/(3*X['ab'])-X['ab']

    absorb_coef=(990/650)*(990/650)*(990/650)*(990/650)
    r=650
    X['absorb']=absorb_coef*X['rho']
    for k in range(0,35):
        name= str(r)+'_od'
        name2=str(r)+'_od2'
        name1=str(r)+'_od1'
        X[name]=-np.log(X[str(r)+'_dst']/X[str(r)+'_src'])
        X[name2]= 223.3+0.05624*np.power(X[name],0.8)+ 0.001245*r*r - 0.9025*r
        X[name1]=X[name]/(X[name2]*X['rho'])
        r+=10  
    X.replace([np.inf, -np.inf], np.nan,inplace=True)
    X.interpolate(axis=1, inplace=True)
    X.fillna(method='bfill',inplace=True)
    #X.drop(src_columns,axis=1,inplace=True)
   # X.drop(dst_columns,axis=1,inplace=True)

        
        
    

In [327]:
train_X.head()

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,960_od1,970_od,970_od2,970_od1,980_od,980_od2,980_od1,990_od,990_od2,990_od1
id,,,,,,,,,,,,,,,,,,,,,
0,25.0,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,0.96333,...,0.003155,38.780756,520.344893,0.002981,37.100948,535.560869,0.002771,34.773339,551.011206,0.002524
1,10.0,0.00000,0.00000,0.01813,0.00000,0.00000,0.01974,0.00321,0.00000,0.00000,...,0.003927,18.661584,519.880026,0.003590,17.994615,535.115753,0.003363,17.828324,550.613052,0.003238
2,25.0,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,0.25584,...,28.443879,30.813912,33.183945,35.553978,37.924011,535.578805,0.002832,37.699338,551.075417,0.002736
3,10.0,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,0.79865,...,0.004152,19.754746,519.907262,0.003800,19.365963,535.150111,0.003619,18.860697,550.639010,0.003425
4,15.0,1.01521,1.00872,0.98930,0.98874,1.01773,1.01632,1.00009,0.98217,1.01564,...,0.003937,29.634120,520.141712,0.003798,29.647138,535.394509,0.003692,29.662270,550.896355,0.003590


In [328]:
def train_model(x_data, y_data, r, k=5):
    models=[]
    
    k_fold = KFold(n_splits=k, shuffle=True, random_state=123)
    
    for train_idx, val_idx in k_fold.split(x_data):
        x_train, y_train = x_data.iloc[train_idx], y_data[train_idx]
        x_val, y_val = x_data.iloc[val_idx], y_data[val_idx]
    
        d_train = xgb.DMatrix(data = x_train, label = y_train)
        d_val = xgb.DMatrix(data = x_val, label = y_val)
        
        wlist = [(d_train, 'train'), (d_val, 'eval')]
        
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'mae',
            'seed':777
            }
        
        model = xgb.train(params=params, dtrain=d_train, num_boost_round=r, verbose_eval=r, evals=wlist)
        models.append(model)
    
    return models







In [329]:
models = {}
#for label in train_Y.columns[range(2)]:
for label in train_Y.columns:
    print('train column : ', label)
    models[label] = train_model(train_X, train_Y[label],r=100)

train column :  hhb
[0]	train-mae:5.26494	eval-mae:5.24360
[99]	train-mae:0.25029	eval-mae:0.81225
[0]	train-mae:5.27133	eval-mae:5.22464
[99]	train-mae:0.23879	eval-mae:0.77604
[0]	train-mae:5.26438	eval-mae:5.26769
[99]	train-mae:0.24071	eval-mae:0.78793
[0]	train-mae:5.25909	eval-mae:5.30563
[99]	train-mae:0.24952	eval-mae:0.81065
[0]	train-mae:5.25668	eval-mae:5.30683
[99]	train-mae:0.23375	eval-mae:0.80075
train column :  hbo2
[0]	train-mae:2.46038	eval-mae:2.47480
[99]	train-mae:0.20655	eval-mae:0.60325
[0]	train-mae:2.46009	eval-mae:2.47434
[99]	train-mae:0.20677	eval-mae:0.60090
[0]	train-mae:2.46920	eval-mae:2.42796
[99]	train-mae:0.20395	eval-mae:0.60949
[0]	train-mae:2.45957	eval-mae:2.47788
[99]	train-mae:0.19706	eval-mae:0.60662
[0]	train-mae:2.46431	eval-mae:2.45680
[99]	train-mae:0.19492	eval-mae:0.62432
train column :  ca
[0]	train-mae:5.99003	eval-mae:6.08417
[99]	train-mae:0.69466	eval-mae:1.96914
[0]	train-mae:5.99177	eval-mae:6.00726
[99]	train-mae:0.64665	eval-mae:

In [ ]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

for col in models:
    preds_train = []
    preds_test = []
    for model in models[col]:
        preds_train.append(model.predict(xgb.DMatrix(train_X)))
        preds_test.append(model.predict(xgb.DMatrix(test_X)))

    pred_train = np.mean(preds_train, axis=0)
    pred_test = np.mean(preds_test, axis=0)
    
    df_train[col] = pred_train
    df_test[col] = pred_test

    sample_submission[col] = pred_test
    sample_submission.to_csv('data_0620.csv',index=False)


In [ ]:
# Function for getting train2_Ys, test_Ys, and coefficients of mapping X -> Ys
def X_Ys_training(train1_X, train1_Ys, train2_X, test_X, day_index, night_index, multi=False):
    # multi_model에서 day, night index를 담는 grid. single model에서는 활용하지 않음.
    if multi:
        # day-night 경계에서 보다 seamless한 추정을 하도록 multiple index에 대해 training한 후 평균을 냄. 
        # multiple index는 day_index, night_index 기준으로 앞뒤로 30분, 60분 지점들의 combination을 meshgrid 형태로 저장함. (5*5=25 models)
        day_grid = np.meshgrid(np.arange(day_index-6, day_index+7, 3), np.arange(night_index-6, night_index+7, 3))[0].reshape(-1)
        night_grid = np.meshgrid(np.arange(day_index-6, day_index+7, 3), np.arange(night_index-6, night_index+7, 3))[1].reshape(-1)
        n_models = day_grid.shape[0]  # n_models=25
    else:
        day_grid, night_grid = [day_index], [night_index]
        n_models = 1
    
    # Ys->Y18 학습에 필요한 train2_Ys, test_Ys 결과 값을 저장할 DataFrame
    train2_Ys = pd.DataFrame(np.zeros((train2_X.shape[0], train1_Ys.shape[1])), index=train2_X.index, columns=train1_Ys.columns)
    test_Ys = pd.DataFrame(np.zeros((test_X.shape[0], train1_Ys.shape[1])), index=test_X.index, columns=train1_Ys.columns)
    
    # Y00부터 Y17까지의 X->Y mapping의 coefficients를 저장하는 리스트. 각 coefficients는 index가 time(0~144)이고 columns가 X(X00, X02, ... , neg_insol)인 DataFrame.
    X_Ys_coefs_list = []
    for i in range(len(train1_Ys.columns)):
        coefs_df = pd.DataFrame(np.zeros((144, train1_X.shape[1])), index=np.arange(144), columns=train1_X.columns)
        coefs_df['Ys_intercept'] = 0  # X->Ys에 대한 intercept column
        X_Ys_coefs_list.append(coefs_df)
    
    # Ys를 구하는 model은 효율을 위해 LassoCV가 아닌 Lasso를 적용.
    lasso = Lasso(alpha=0.05)
    
    # n_models개의 Day, night index의 조합에 대해 각각 model을 학습해 coefficients를 구함. (5*5=25 models)
    for day, night in zip(day_grid, night_grid):
        tr1_day_ids, tr1_night_ids = day_night_split(train1_X, day, night)
        day_train1_X, night_train1_X = train1_X.loc[tr1_day_ids], train1_X.loc[tr1_night_ids]
        
        tr2_day_ids, tr2_night_ids = day_night_split(train2_X, day, night)
        day_train2_X, night_train2_X = train2_X.loc[tr2_day_ids], train2_X.loc[tr2_night_ids]
                
        ts_day_ids, ts_night_ids = day_night_split(test_X, day, night)
        day_test_X, night_test_X = test_X.loc[ts_day_ids], test_X.loc[ts_night_ids]
        
        # 낮 시간대에서 X12(humidity), X20(humidity)가 noisy 하기 때문에 drop.
        for day_X in [day_train1_X, day_train2_X, day_test_X]:
            day_X.drop(['X12', 'X20'], axis=1, inplace=True)
            
        # 밤 시간대에서 X31(temperature), X38(humidity)가 noisy 하기 때문에 drop
        for night_X in [night_train1_X, night_train2_X, night_test_X]:
            night_X.drop(['X31', 'X38'], axis=1, inplace=True)
        
        # Y00부터 Y17까지 Loop
        for i, c_name in enumerate(train1_Ys.columns):
            # ----------- Day training -----------
            day_train1_Y = train1_Ys.loc[tr1_day_ids, c_name]
            lasso.fit(day_train1_X, day_train1_Y)
            
            # day_time에 대한 coefficients를 X_Ys_coefs_list에 저장. n개의 model에서의 coefficients들의 평균을 얻기 위해 n_models로 나누어줌.
            day_time = range(day+1, night+1)
            coefs = np.tile(lasso.coef_, (len(day_time), 1)) / n_models
            X_Ys_coefs_list[i].loc[day_time, day_train1_X.columns] += coefs
            X_Ys_coefs_list[i].loc[day_time, 'Ys_intercept'] += lasso.intercept_ / n_models
            
            # train2_Ys, test_Ys Predict. n개의 model에서의 prediction 값의 평균을 얻기 위해 n_models로 나누어줌.
            train2_Ys.loc[tr2_day_ids, c_name] += lasso.predict(day_train2_X) / n_models
            test_Ys.loc[ts_day_ids, c_name] += lasso.predict(day_test_X) / n_models
            
            # ----------- Night training -----------
            # Same as Day training
            night_train1_Y = train1_Ys.loc[tr1_night_ids, c_name]
            lasso.fit(night_train1_X, night_train1_Y)
            
            night_time = list(range(0, day+1)) + list(range(night+1, 144))
            coefs = np.tile(lasso.coef_, (len(night_time), 1)) / n_models
            X_Ys_coefs_list[i].loc[night_time, night_train1_X.columns] += coefs
            X_Ys_coefs_list[i].loc[night_time, 'Ys_intercept'] += lasso.intercept_ / n_models
            
            train2_Ys.loc[tr2_night_ids, c_name] += lasso.predict(night_train2_X) / n_models
            test_Ys.loc[ts_night_ids, c_name] += lasso.predict(night_test_X) / n_models
    
    return train2_Ys, test_Ys, X_Ys_coefs_list


# train2_Y18에 대해 correlation 값이 상위 n개인 Y들을 구함 -> Ys_high_correlated
def high_corr_Ys(train_Y, n=None):
    if n==None:
        n = train_Y.shape[1]
        
    corrs = pd.DataFrame(index=train_Y.columns, columns=['corr'])
    for c in train_Y:
        corrs.loc[c, 'corr'] = np.corrcoef(train_Y[c], train2_Y18)[0,1]
    corrs = corrs.sort_values(by='corr', ascending=False)
    
    Ys_high_correlated = corrs[corrs.columns[:n]].index
 
    return Ys_high_correlated


# Function for getting coefficients of mapping Ys -> Y18
def Ys_Y18_training(train1_X, train2_Ys, train2_Y18, test_Ys, X_Ys_coefs_list, day_index, night_index):
    tr2_day_ids, tr2_night_ids = day_night_split(train2_Ys, day_index, night_index)
    day_train2_Ys, night_train2_Ys = train2_Ys.loc[tr2_day_ids], train2_Ys.loc[tr2_night_ids]
    day_train2_Y18, night_train2_Y18 = train2_Y18.loc[tr2_day_ids], train2_Y18.loc[tr2_night_ids]
    
    # X->Y18 mapping의 coefficients를 저장하는 DataFrame. index는 time(0~144)이고 columns는 X(X00, X02, ... , neg_insol)임.
    X_Y18_coefs = pd.DataFrame(np.zeros((144, train1_X.shape[1])), index=np.arange(144), columns=train1_X.columns)
    X_Y18_coefs['Ys_intercept'] = 0  # X->Ys 에서의 intercept에 대한 coefficient
    X_Y18_coefs['Y18_intercept'] = 0  # Ys->Y18 에서의 intercept에 대한 coefficient 
    
    # Day, Night index에 따라 optimal alpha 값이 달라지므로, LassoCV를 이용함.
    model = LassoCV(eps=0.01, n_alphas=40, cv=5, n_jobs=-1)
    
    # Day training
    day_Ys_high_correlated = high_corr_Ys(day_train2_Ys, day_train2_Y18, n=10)  # correlation이 높음 10개의 Y로 training을 진행.
    day_train2_Ys = day_train2_Ys[day_Ys_high_correlated]
    model.fit(day_train2_Ys, day_train2_Y18)
    
    # Ys->Y18에 더함.
    # 예를 들어, Ys->Y18의 coefficients가 {'Y00': 0, Y01': 1.2, 'Y02': 0.1} 이라고 하면 X_Y18_coefs는 아래와 같이 구할 수 있음.
    # X_Y18_coefs = (X_Ys_coefs_list[0] * 0) + (X_Ys_coefs_list[1] * 1.2) + (X_Ys_coefs_list[2] * 0.1) 
    # day와 night에 각각에 대해 X_Y18_coefs를 계산한다.
    Ys_Y18_coefs = pd.Series(model.coef_, index=day_train2_Ys.columns)
    Ys_Y18_coefs = Ys_Y18_coefs.loc[Ys_Y18_coefs!=0]
    day_time = np.arange(day_index+1, night_index+1)
    for Y in Ys_Y18_coefs.index:
        idx = np.where(train2_Ys.columns==Y)[0][0]
        X_Y18_coefs.loc[day_time, X_Y18_coefs.columns[:-1]] += X_Ys_coefs_list[idx].loc[day_time] * Ys_Y18_coefs.loc[Y]
        X_Y18_coefs.loc[day_time, 'Y18_intercept'] = model.intercept_
    
    # Night training (Same as Day training)
    night_Ys_high_correlated = high_corr_Ys(night_train2_Ys, night_train2_Y18, n=10)
    night_train2_Ys = night_train2_Ys[night_Ys_high_correlated]
    model.fit(night_train2_Ys, night_train2_Y18)
    
    Ys_Y18_coefs = pd.Series(model.coef_, index=night_train2_Ys.columns)
    Ys_Y18_coefs = Ys_Y18_coefs.loc[Ys_Y18_coefs!=0]
    night_time = list(range(0, day_index+1)) + list(range(night_index+1, 144))
    for Y in Ys_Y18_coefs.index:
        idx = np.where(train2_Ys.columns==Y)[0][0]
        X_Y18_coefs.loc[night_time, X_Y18_coefs.columns[:-1]] += X_Ys_coefs_list[idx].loc[night_time] * Ys_Y18_coefs.loc[Y]
        X_Y18_coefs.loc[night_time, 'Y18_intercept'] = model.intercept_
    
    return X_Y18_coefs


# Function for predicting Y18 with X_Y18_coefs. 
def predict_Y18(train1_X, target_X, X_Y18_coefs):
    scaler = StandardScaler().fit(train1_X)
    target_X = pd.DataFrame(scaler.transform(target_X), index=target_X.index, columns=target_X.columns)
  
    # intercept는 변수가 아니므로 1.0으로 고정함. (X_Y18_coefs에 저장된 값을 그대로 사용하기 위해) 
    target_X['Ys_intercept'] = 1.0
    target_X['Y18_intercept'] = 1.0
    
    target_Y18 = pd.Series(np.zeros((target_X.shape[0])), index=target_X.index)

    # time 0부터 143까지, target_X의 시간 t의 데이터와 X_Y18_coefs의 시간 t에 대한 coefficients를 inner product.
    for t in range(144):
        Y_pred = target_X.loc[(target_X.index%144)==t].dot(X_Y18_coefs.loc[t].values)
        target_Y18.loc[Y_pred.index] = Y_pred
    
    target_X.drop(['Ys_intercept', 'Y18_intercept'], axis=1, inplace=True)
    
    return target_Y18


# model for single day, night index 
def single_model(train1_X, train1_Ys, train2_X, train2_Y18, test_X, day_night_index, multi=False):
    day_index, night_index = day_night_index
    print('day_index : %s, night_index : %s' % (day_index, night_index))
    # ----------------------- Standard Scaler -----------------------
    scaler = StandardScaler().fit(train1_X)
    train1_X = pd.DataFrame(scaler.transform(train1_X), index=train1_X.index, columns=train1_X.columns)
    train2_X = pd.DataFrame(scaler.transform(train2_X), index=train2_X.index, columns=train2_X.columns)
    test_X = pd.DataFrame(scaler.transform(test_X), index=test_X.index, columns=test_X.columns)

    # ----------------------- Predict Y00-Y17 -----------------------
    train2_Ys, test_Ys, X_Ys_coefs_list = X_Ys_training(train1_X, train1_Ys, train2_X, test_X, day_index, night_index, multi)
    
    # ----------------------- Predict Y18 -----------------------
    X_Y18_coefs = Ys_Y18_training(train1_X, train2_Ys, train2_Y18, test_Ys, X_Ys_coefs_list, day_index, night_index)
    
    # multi인 경우엔 predict를 하지 않고 coefs만 return함.
    if multi:
        return X_Y18_coefs
    else:
        train2_Y18_pred = predict_Y18(train1_X, train2_X, X_Y18_coefs)
        test_Y18 = predict_Y18(train1_X, test_X, X_Y18_coefs)
        return train2_Y18_pred, test_Y18,  X_Y18_coefs
    
    
single_train2_Y18_pred, single_test_Y18, single_X_Y18_coefs =\
single_model(train1_X, train1_Ys, train2_X, train2_Y18, test_X, day_night_index=(33, 100), multi=False)

# single model training error
mean_squared_error(single_train2_Y18_pred, train2_Y18)

fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(x=train2_Y18.index, y=train2_Y18, ax=ax, c='k')
sns.lineplot(x=train2_Y18.index, y=single_train2_Y18_pred, ax=ax, c='r')

In [ ]:

def multi_model(day_start, day_end, night_start, night_end, predict=True, **single_model_params):
    # day, night index의 grid. 총 ((day_end - day_start) / 2) * ((night_end - night_start) / 2) 개의 models.
    day_grid = np.meshgrid(np.arange(day_start, day_end, 2), np.arange(night_start, night_end, 2))[0].reshape(-1)
    night_grid = np.meshgrid(np.arange(day_start, day_end, 2), np.arange(night_start, night_end, 2))[1].reshape(-1)
    n_models = day_grid.shape[0]
    
    # Multiprocessing
    pool = ThreadPool(8)
    X_Y18_coefs_list = pool.map(lambda grid: single_model(day_night_index=grid, multi=True, **single_model_params), zip(day_grid, night_grid))
    
    # list에 저장된 multiprocessing의 결과(coefficients for each day, night combination)의 평균값을 구함.
    multi_X_Y18_coefs = pd.DataFrame(np.zeros((144, train1_X.shape[1])), index=np.arange(144), columns=train1_X.columns)
    multi_X_Y18_coefs['Ys_intercept'] = 0
    multi_X_Y18_coefs['Y18_intercept'] = 0
    for coefs in X_Y18_coefs_list:
        multi_X_Y18_coefs += coefs
    multi_X_Y18_coefs /= n_models
    
    # Predict Y18
    if predict:
        train2_Y18 = predict_Y18(train1_X, train2_X, multi_X_Y18_coefs)
        test_Y18 = predict_Y18(train1_X, test_X, multi_X_Y18_coefs)
        return train2_Y18, test_Y18, multi_X_Y18_coefs
    else:
        return multi_X_Y18_coefs

In [ ]:
%%time
# day_start, day_ends, night_start, night_end 값은 리더보드 점수가 가장 높은 것으로 선택함.
train2_Y18_pred, test_Y18, multi_X_Y18_coefs = multi_model(day_start=21, day_end=45, night_start=91, night_end=109, predict=True,
                                                           train1_X=train1_X, train1_Ys=train1_Ys, train2_X=train2_X, train2_Y18=train2_Y18, test_X=test_X)

In [ ]:
mean_squared_error(train2_Y18_pred, train2_Y18)

# Visualization : train2_Y18 prediction 
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(x=train2_Y18.index, y=train2_Y18, ax=ax, c='k')
sns.lineplot(x=train2_Y18.index, y=train2_Y18_pred, ax=ax, c='r')

ValueError: Cannot describe a DataFrame without columns

In [27]:
df_train=pd.concat([train_X,df_train],axis=1)
df_test=pd.concat([test_X,df_test],axis=1)

In [28]:
models_2 = {}
for label in train_Y.columns[range(2,4)]:
    print('train column : ', label)
    model = LinearRegression()
    model.fit(df_train, train_Y)
    r_sq = model.score(df_train,train_Y)
    models_2[label] = train_model(df_train, train_Y[label],r=1000)

train column :  ca
[0]	train-mae:5.99076	eval-mae:6.08719
[999]	train-mae:0.00169	eval-mae:2.02914
[0]	train-mae:5.99972	eval-mae:6.02855
[999]	train-mae:0.00143	eval-mae:2.06457
[0]	train-mae:5.99876	eval-mae:6.04462


KeyboardInterrupt: 